In [93]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
import numpy as np
pd.options.display.max_rows = 20

# Data read and prep

The first step we take is to read in our data.

In [3]:
alltex = pd.read_csv("alltexts copy.csv")
alltex = alltex.drop(alltex.columns[0], axis=1) #drop the first column, which is just the row numbers from the csv
alltex

,cf,extent,form,gw,id_text,lang,line_no,line_ref,pos,status,text_name,version
0,dubsaŋ,NaN,dub-saŋ-ta,first,c.0.1.1,sux,1,1,AJ,NaN,Ur III catalogue from Nibru (N1),NaN
1,Enki,NaN,{d}en-ki,1,c.0.1.1,sux,2,2,DN,NaN,Ur III catalogue from Nibru (N1),NaN
2,unu,NaN,unu₂,dwelling,c.0.1.1,sux,2,2,N,NaN,Ur III catalogue from Nibru (N1),NaN
3,gal,NaN,gal,big,c.0.1.1,sux,2,2,V/i,NaN,Ur III catalogue from Nibru (N1),NaN
4,ed,NaN,im-ed₃,ascend,c.0.1.1,sux,2,2,V/i,NaN,Ur III catalogue from Nibru (N1),NaN
5,anzag,NaN,an-zag-še₃,horizon,c.0.1.1,sux,3,3,N,NaN,Ur III catalogue from Nibru (N1),NaN
6,anŋi,NaN,an-ŋi₆,eclipse,c.0.1.1,sux,4,4,N,NaN,Ur III catalogue from Nibru (N1),NaN
7,zu,NaN,zu,know,c.0.1.1,sux,4,4,V/t,NaN,Ur III catalogue from Nibru (N1),NaN
8,ama,NaN,ama,mother,c.0.1.1,sux,4,4,N,NaN,Ur III catalogue from Nibru (N1),NaN
9,tu,NaN,tu₆,incantation,c.0.1.1,sux,4,4,N,NaN,Ur III catalogue from Nibru (N1),NaN


Next, we filter out languages we don't care about. To do this, we look at the 'lang' column, and select only those rows where the 'lang' value starts with 'sux'

In [4]:
alltex = alltex.loc[[str(i)[:3] == 'sux' for i in alltex.lang], :]
alltex

,cf,extent,form,gw,id_text,lang,line_no,line_ref,pos,status,text_name,version
0,dubsaŋ,NaN,dub-saŋ-ta,first,c.0.1.1,sux,1,1,AJ,NaN,Ur III catalogue from Nibru (N1),NaN
1,Enki,NaN,{d}en-ki,1,c.0.1.1,sux,2,2,DN,NaN,Ur III catalogue from Nibru (N1),NaN
2,unu,NaN,unu₂,dwelling,c.0.1.1,sux,2,2,N,NaN,Ur III catalogue from Nibru (N1),NaN
3,gal,NaN,gal,big,c.0.1.1,sux,2,2,V/i,NaN,Ur III catalogue from Nibru (N1),NaN
4,ed,NaN,im-ed₃,ascend,c.0.1.1,sux,2,2,V/i,NaN,Ur III catalogue from Nibru (N1),NaN
5,anzag,NaN,an-zag-še₃,horizon,c.0.1.1,sux,3,3,N,NaN,Ur III catalogue from Nibru (N1),NaN
6,anŋi,NaN,an-ŋi₆,eclipse,c.0.1.1,sux,4,4,N,NaN,Ur III catalogue from Nibru (N1),NaN
7,zu,NaN,zu,know,c.0.1.1,sux,4,4,V/t,NaN,Ur III catalogue from Nibru (N1),NaN
8,ama,NaN,ama,mother,c.0.1.1,sux,4,4,N,NaN,Ur III catalogue from Nibru (N1),NaN
9,tu,NaN,tu₆,incantation,c.0.1.1,sux,4,4,N,NaN,Ur III catalogue from Nibru (N1),NaN


Next, we'd like to filter out the rows that have nulls in the cf, gw, or pos columns. In other words, we keep columns that *don't* have NA in gw AND *don't* have NA in form AND *don't* have NA in pos. 

In [5]:
prefilter = alltex #we'll use this later

In [6]:
alltex = alltex.loc[np.logical_and(~pd.isnull(alltex.gw),
                         np.logical_and(~pd.isnull(alltex.form),
                                       ~pd.isnull(alltex.pos))
                         )]
alltex

,cf,extent,form,gw,id_text,lang,line_no,line_ref,pos,status,text_name,version
0,dubsaŋ,NaN,dub-saŋ-ta,first,c.0.1.1,sux,1,1,AJ,NaN,Ur III catalogue from Nibru (N1),NaN
1,Enki,NaN,{d}en-ki,1,c.0.1.1,sux,2,2,DN,NaN,Ur III catalogue from Nibru (N1),NaN
2,unu,NaN,unu₂,dwelling,c.0.1.1,sux,2,2,N,NaN,Ur III catalogue from Nibru (N1),NaN
3,gal,NaN,gal,big,c.0.1.1,sux,2,2,V/i,NaN,Ur III catalogue from Nibru (N1),NaN
4,ed,NaN,im-ed₃,ascend,c.0.1.1,sux,2,2,V/i,NaN,Ur III catalogue from Nibru (N1),NaN
5,anzag,NaN,an-zag-še₃,horizon,c.0.1.1,sux,3,3,N,NaN,Ur III catalogue from Nibru (N1),NaN
6,anŋi,NaN,an-ŋi₆,eclipse,c.0.1.1,sux,4,4,N,NaN,Ur III catalogue from Nibru (N1),NaN
7,zu,NaN,zu,know,c.0.1.1,sux,4,4,V/t,NaN,Ur III catalogue from Nibru (N1),NaN
8,ama,NaN,ama,mother,c.0.1.1,sux,4,4,N,NaN,Ur III catalogue from Nibru (N1),NaN
9,tu,NaN,tu₆,incantation,c.0.1.1,sux,4,4,N,NaN,Ur III catalogue from Nibru (N1),NaN


Next, I filter out "X" and "(X)". We want rows where gw is *not* X or (X) AND form is *not* X or (X) AND pos is *not* X or (X).

In [7]:
xstrings = ["X", "(X)"]

In [8]:
alltex = alltex.loc[np.logical_and(~np.in1d(alltex.gw, xstrings) ,
                         np.logical_and(~np.in1d(alltex.form, xstrings),
                                       ~np.in1d(alltex.pos, xstrings))
                         )]
alltex

,cf,extent,form,gw,id_text,lang,line_no,line_ref,pos,status,text_name,version
0,dubsaŋ,NaN,dub-saŋ-ta,first,c.0.1.1,sux,1,1,AJ,NaN,Ur III catalogue from Nibru (N1),NaN
1,Enki,NaN,{d}en-ki,1,c.0.1.1,sux,2,2,DN,NaN,Ur III catalogue from Nibru (N1),NaN
2,unu,NaN,unu₂,dwelling,c.0.1.1,sux,2,2,N,NaN,Ur III catalogue from Nibru (N1),NaN
3,gal,NaN,gal,big,c.0.1.1,sux,2,2,V/i,NaN,Ur III catalogue from Nibru (N1),NaN
4,ed,NaN,im-ed₃,ascend,c.0.1.1,sux,2,2,V/i,NaN,Ur III catalogue from Nibru (N1),NaN
5,anzag,NaN,an-zag-še₃,horizon,c.0.1.1,sux,3,3,N,NaN,Ur III catalogue from Nibru (N1),NaN
6,anŋi,NaN,an-ŋi₆,eclipse,c.0.1.1,sux,4,4,N,NaN,Ur III catalogue from Nibru (N1),NaN
7,zu,NaN,zu,know,c.0.1.1,sux,4,4,V/t,NaN,Ur III catalogue from Nibru (N1),NaN
8,ama,NaN,ama,mother,c.0.1.1,sux,4,4,N,NaN,Ur III catalogue from Nibru (N1),NaN
9,tu,NaN,tu₆,incantation,c.0.1.1,sux,4,4,N,NaN,Ur III catalogue from Nibru (N1),NaN


At this point, we're ready to start lemmatizing, or creating the "terms". Terms look like this for each word: 

cf + '[' + gw + ']' + pos 

To do this, we'll paste together the appropriate column entries for each row, and create a new column called "Term".

In [9]:
alltex['Term'] = [str(alltex.cf[i]) + "[" + str(alltex.gw[i]) + "]" + str(alltex.pos[i]) for i in alltex.index]
alltex

C:\Users\ckaiw.LAPTOP-THCC7RLJ\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,cf,extent,form,gw,id_text,lang,line_no,line_ref,pos,status,text_name,version,Term
0,dubsaŋ,NaN,dub-saŋ-ta,first,c.0.1.1,sux,1,1,AJ,NaN,Ur III catalogue from Nibru (N1),NaN,dubsaŋ[first]AJ
1,Enki,NaN,{d}en-ki,1,c.0.1.1,sux,2,2,DN,NaN,Ur III catalogue from Nibru (N1),NaN,Enki[1]DN
2,unu,NaN,unu₂,dwelling,c.0.1.1,sux,2,2,N,NaN,Ur III catalogue from Nibru (N1),NaN,unu[dwelling]N
3,gal,NaN,gal,big,c.0.1.1,sux,2,2,V/i,NaN,Ur III catalogue from Nibru (N1),NaN,gal[big]V/i
4,ed,NaN,im-ed₃,ascend,c.0.1.1,sux,2,2,V/i,NaN,Ur III catalogue from Nibru (N1),NaN,ed[ascend]V/i
5,anzag,NaN,an-zag-še₃,horizon,c.0.1.1,sux,3,3,N,NaN,Ur III catalogue from Nibru (N1),NaN,anzag[horizon]N
6,anŋi,NaN,an-ŋi₆,eclipse,c.0.1.1,sux,4,4,N,NaN,Ur III catalogue from Nibru (N1),NaN,anŋi[eclipse]N
7,zu,NaN,zu,know,c.0.1.1,sux,4,4,V/t,NaN,Ur III catalogue from Nibru (N1),NaN,zu[know]V/t
8,ama,NaN,ama,mother,c.0.1.1,sux,4,4,N,NaN,Ur III catalogue from Nibru (N1),NaN,ama[mother]N
9,tu,NaN,tu₆,incantation,c.0.1.1,sux,4,4,N,NaN,Ur III catalogue from Nibru (N1),NaN,tu[incantation]N


Now, with these terms, we can filter out stop words and "stop docs".
* Stop word: A term to common in the language to be useful for classification; examples in English include the articles (a, an, the), as they are used in nearly every document and thus are not useful for distinguishing between documents.
* "Stop doc": A document that we would not like to include for whatever reason, such as being too short

The things we want to filter out could change, so I'll write a function to do this filtering.

In [10]:
def filter(table, stopwords = [], stopdocs = []):
    """
    Parameters:
        table: Pandas dataframe
            The table to modify/filter.
        stopwords: Arraylike, Strings
            A collection of strings of terms to filter out
        stopdocs: Arraylike, Strings
    Returns a table without the terms and documents specified
    """
    return table.loc[np.logical_and(~np.in1d(table.Term, stopwords), ~np.in1d(table.id_text, stopdocs))]

In [12]:
stopwords = np.array(pd.read_csv("stopwords_top21.txt", header=None)[0])
stopdocs = np.array(pd.read_csv("stopdocuments_less50.txt", header = None)[0])
stopwords,stopdocs

(array(['dug[speak]V/t', 'ki[place]N', 'šu[hand]N', 'gal[big]V/i',
        'lu[person]N', 'e[house]N', 'ŋar[place]V/t', 'šag[heart]N',
        'kur[mountain]N', 'lugal[king]N', 'ud[sun]N', 'Enlil[1]DN',
        'igi[eye]N', 'e[leave]V/i', 'kug[pure]V/i', 'an[sky]N',
        'saŋ[head]N', 'ak[do]V/t', 'gub[stand]V/i', 'en[lord]N',
        'ŋal[be]V/i'], dtype=object),
 array(['c.0.2.05', 'c.0.2.12', 'c.2.4.1.8', 'c.2.4.2.12', 'c.2.4.2.23',
        'c.2.4.4.5', 'c.2.4.4.9', 'c.2.5.2.3', 'c.2.5.4.16', 'c.2.5.5.8',
        'c.2.6.9.a', 'c.2.8.2.5', 'c.2.8.3.4', 'c.2.8.3.7', 'c.2.8.3.8',
        'c.2.99.b', 'c.2.99.c', 'c.2.99.d', 'c.3.1.10', 'c.3.1.13.1',
        'c.3.2.01', 'c.3.3.06', 'c.3.3.07', 'c.3.3.12', 'c.3.3.20',
        'c.3.3.27', 'c.4.08.12', 'c.4.08.13', 'c.4.08.22', 'c.4.13.b',
        'c.4.13.d', 'c.4.19.4', 'c.4.22.3', 'c.4.27.a', 'c.4.30.1',
        'c.5.7.3', 'c.5.7.a', 'c.6.2.4'], dtype=object))

In [13]:
alltex = filter(alltex, stopwords=stopwords, stopdocs = stopdocs)
alltex

,cf,extent,form,gw,id_text,lang,line_no,line_ref,pos,status,text_name,version,Term
0,dubsaŋ,NaN,dub-saŋ-ta,first,c.0.1.1,sux,1,1,AJ,NaN,Ur III catalogue from Nibru (N1),NaN,dubsaŋ[first]AJ
1,Enki,NaN,{d}en-ki,1,c.0.1.1,sux,2,2,DN,NaN,Ur III catalogue from Nibru (N1),NaN,Enki[1]DN
2,unu,NaN,unu₂,dwelling,c.0.1.1,sux,2,2,N,NaN,Ur III catalogue from Nibru (N1),NaN,unu[dwelling]N
4,ed,NaN,im-ed₃,ascend,c.0.1.1,sux,2,2,V/i,NaN,Ur III catalogue from Nibru (N1),NaN,ed[ascend]V/i
5,anzag,NaN,an-zag-še₃,horizon,c.0.1.1,sux,3,3,N,NaN,Ur III catalogue from Nibru (N1),NaN,anzag[horizon]N
6,anŋi,NaN,an-ŋi₆,eclipse,c.0.1.1,sux,4,4,N,NaN,Ur III catalogue from Nibru (N1),NaN,anŋi[eclipse]N
7,zu,NaN,zu,know,c.0.1.1,sux,4,4,V/t,NaN,Ur III catalogue from Nibru (N1),NaN,zu[know]V/t
8,ama,NaN,ama,mother,c.0.1.1,sux,4,4,N,NaN,Ur III catalogue from Nibru (N1),NaN,ama[mother]N
9,tu,NaN,tu₆,incantation,c.0.1.1,sux,4,4,N,NaN,Ur III catalogue from Nibru (N1),NaN,tu[incantation]N
10,zu,NaN,zu-ke₄,know,c.0.1.1,sux,4,4,V/t,NaN,Ur III catalogue from Nibru (N1),NaN,zu[know]V/t


At this point, we are ready to create a tfidf matrix.

In [14]:
np.unique(alltex.Term).shape #this gives us an idea of how many columns we should see in the tfidf 

(4302,)

# TFIDF

We'd like to numerically represent our data in a useful way. For this purpose, we create a document-term matrix: a matrix with terms as the column labels, and documents as the rows. More generally, a document is split up into "tokens"- words, or group of words (ie, ngrams) that can help classify a document. These tokens allow us to basically represent documents as points in an N-d space, where N is the number of unique tokens used across documents. For example, if I had document A = "hello hello bye there" and Document B = "there there hello hello", I could represent the first as the point (2, 1, 1) and the second as (2, 0, 2), with the order being (# of hellos, # of byes, # of theres). One could imagine, then, a matrix with each document as a row and each term as a column; the rows of this matrix would then basically look like the lists of numbers just given. The length of these rows is the number o unique terms across all the documents in our corpus.

In a document term matrix (DTM)The value in the cell at position (document, term) will have a number corresponding to how frequently that word appears in the document. However, if the word is very common across all our documents, it won't be as helpful in classification. So we'd like to "penalize" words that appear too frequently, where "too frequently" is based on how many documents it appears in. Wanting a measure these two things
* How often some term t appears in a particular document d
* How often this term t appears in all documents (and penalize for this)
leads us to the TFIDF (term frequency inverse document frequency measure).

We calculate a TFIDF by the formula
$$
tfidf(t, d, D) = tf(t,d)\cdot idf(t,D)
$$
where $t$ is the term, $d$ is the document, and $D$ is the corpus.
Here, $tf(t,d)$ is just going to be a function returning the raw count of term t in document d. The idf will be  
$$
log\frac{1 + N}{1 + n_t} + 1
$$
where N is the number of documents in the corpus and n_t is the number of docs where the term appears (formally $|{d \in D: t \in d}|$). The +1s in the log are to safeguard against log(0) or division by 0 errors, while the +1 outside ensures a positive value.

This means the overall expression for tfidf is
$$
f_{t,d}\cdot ln(\frac{N}{n_t})
$$
The full workings of the following procedure are documented here: http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction

To create this tfidf, we use sklearn's vectorizer. But before doing that, we need to represent our corpus as a list of documents, which it currently is not (it's currently split up in our dataframe). To do this, we aggregate:

In [15]:
grouped = alltex.groupby("id_text").aggregate(" ".join) #this line of code lumps together all documents' terms together, and joins them as a single space separated string
grouped

,cf,form,gw,lang,line_no,pos,text_name,Term
id_text,,,,,,,,
c.0.1.1,dubsaŋ Enki unu ed anzag anŋi zu ama tu zu gi ...,dub-saŋ-ta {d}en-ki unu₂ im-ed₃ an-zag-še₃ an-...,first 1 dwelling ascend horizon eclipse know m...,sux sux sux sux sux sux sux sux sux sux sux su...,1 2 2 2 3 4 4 4 4 4 5 5 6 7 7 7 8 8 8 8 9 10 1...,AJ DN N V/i N N V/t N N V/t N V/t N N N V/i N ...,Ur III catalogue from Nibru (N1) Ur III catalo...,dubsaŋ[first]AJ Enki[1]DN unu[dwelling]N ed[as...
c.0.1.2,diŋir šembizida dar kur mete gud banda šerzid ...,diŋir šembi₂-zid-da dar-a nu-kur₂ me-te-bi gud...,deity kohl split different appropriate-thing o...,sux sux sux sux sux sux sux sux sux sux sux su...,1 1 1 2 3 5 5 6 7 7 7 8 9 9 10 10 10 10 11 11 ...,N N V/t V/i N N V/i N N V/i N N N V/i N N NU V...,Ur III catalogue at Yale (Y1) Ur III catalogue...,diŋir[deity]N šembizida[kohl]N dar[split]V/t k...
c.0.2.01,mi niŋdu nin me šar sud nam nun re innin me hu...,mi₂ niŋ₂-du₇-e nin me šar₂-ra su₃-ra₂-še₃ nam₂...,cvne appropriate-thing lady Being 3600 distant...,sux sux sux sux sux sux sux sux sux sux sux su...,2 3 4 4 4 5 6 6 7 8 8 8 9 9 10 11 11 12 12 13 ...,N N N N NU V/i N N DP N N V/i N V/i V/i V/i N ...,OB catalogue from Nibru (N2) OB catalogue from...,mi[cvne]N niŋdu[appropriate-thing]N nin[lady]N...
c.0.2.02,sud nam nun re innin me huš ud huš til hursaŋ ...,su₃-ra₂-še₃ nam₂ nun-e re-a in-nin me huš-a ud...,distant lord prince that lady Being reddish st...,sux sux sux sux sux sux sux sux sux sux sux su...,5 6 6 7 8 8 8 9 9 10 11 12 12 13 13 14 15 15 1...,V/i N N DP N N V/i N V/i V/i N N V/i N N DP DP...,OB catalogue in the Louvre (L) OB catalogue in...,sud[distant]V/i nam[lord]N nun[prince]N re[tha...
c.0.2.03,mu niŋul Waradsin Inab me ŋuruš šir rah enbar ...,mu-ni niŋ₂-ul-še₃ IR₃-{d}suen i₃-na-ab i-me-a ...,name everlasting 1 1 be male song beat reed fr...,sux sux sux sux sux sux sux sux sux sux sux su...,1 1 2 3 3 4 4 4 5 5 6 7 8 9 10 10 10 11 11 12 ...,N N RN SN V/i N N V/t N N N RN V/i N N N V/i Q...,OB catalogue from Urim (U1) OB catalogue from ...,mu[name]N niŋul[everlasting]N Waradsin[1]RN In...
c.0.2.04,nin mulan buršuma niŋziŋal ursaŋ mi meta di me...,nin mul-an-gin₇ bur-šum₂-ma zi-ŋal₂ ur-saŋ-me-...,lady star dowager living-creature hero cvne wh...,sux sux sux sux sux sux sux sux-x-emesal sux s...,1 1 2 3 4 5 6 6 7 7 8 8 8 9 10 10 11 11 12 13 ...,N N N N N N QP V/i QP V/i N N NU V/i J V/i V/i...,OB catalogue from Urim (U2) OB catalogue from ...,nin[lady]N mulan[star]N buršuma[dowager]N niŋz...
c.0.2.06,ursaŋ alim mah mea lu meʾam RI-RI iri re me ga...,ur-saŋ alim mah me-a lu me-a-am nu-mu-e-de₃-RI...,hero bison great where? abundant dear RI-RI ci...,sux sux sux sux sux sux sux sux sux sux sux su...,1 1 1 2 2 3 4 5 6 7 7 8 8 9 9 10 10 11 12 12 1...,N N V/i QP V/i J V N DP N V/i QP N DN DN NU V/...,OB catalogue from Nibru (N3) OB catalogue from...,ursaŋ[hero]N alim[bison]N mah[great]V/i mea[wh...
c.0.2.07,uru2 er balaŋ Nin.X illalum aya illalum balaŋ ...,uru₂ am₃-ma-er₂-ra-bi balaŋ {d}nin-X il-la-lum...,town tears instrument 1 expression cry express...,sux-x-emesal sux sux sux sux sux sux sux sux s...,A1 A1 A2 A2 A4 A4 A4 A5 A5 A6 A6 A6 A7 A7 A7 A...,N N N DN J N J N DN N V/i N NU N DN NU N NU N ...,OB catalogue possibly from Zimbir (B1) OB cata...,uru2[town]N er[tears]N balaŋ[instrument]N Nin....
c.0.2.08,a-ce-er il nir atuku gel-le-ej3 mah uma gig um...,a-še-er il₂ nir a₂-tuku gel-le-eŋ₃-ŋa₂-ŋu₁₀ ma...,lament raise trust powerful to-be-lost great t...,sux-x-emesal sux sux sux sux-x-emesal sux sux ...,A4 A6 B3 B3 B4 B6 B7 B7 B9 B9 B10 B10 B11 B11 ...,N V/t N N V V/i N V/i N V/t N V/i DN N V/i DN ...,OB catalogue from Nibru (N4) OB catalogue from...,a-ce-er[lament]N il[raise]V/t nir[trust]N atuk...


We start off by just making a regular term matrix: just count up the number of times we see a word in a particular document. Sklearn's countvectorizer takes care of this nicely. Below, we create a countvectorizer object. This is essentially like a really flexible tool ()like a Swiss army knife, perhaps) that we've just created in code. The parameters that are passed in sort of "configure" our Swiss army knife to behave in a particular way. Then, in later cells, we use a particular tool in our Swiss army knife to help us accomplish the end goal of creating a DTM. 

Below, we've initialized a count vectorizer object. Here, the second argument is fairly straightforward to understand: when we apply this count vectorizer to something we want to turn into a DTM, we leave the capitalization as is. 
The first is a bit more strange. This is a string that we've set that helps the CV object recognize what we want to be a token (a term, in this case). The default behavior for this object is just to treat all punctuation and whitespace as separators. So, it might just end up separating our grouped terms into separate tokens, which is not idea. This string passed in for token_pattern tells the count vectorizer that our tokens should 1) have at least 1 character, and 2) not contain any spaces. The CV then separates a string passed in into chunk, each chunk fulfiling the criteria set by the string. Basically, this means that we've told the count vectorizer to only separate at spaces and nothing else. This particular string searching is known as a regular experssion (regex), which you can learn more about and play around with here: https://regexr.com/. 

In [145]:
cv = CountVectorizer(token_pattern='[^ ]+', lowercase = False) #initialize a count vectorizer object with correct tokenization for this term format

Then, below, we've called upon one of the "tools" of the Swiss army knife; .fit_transform, which takes in a list of strings, interprets each string as a document, and spits out the DTM based on this interpretation. It splits apart each document based on the toke pattern given above, and doesn't force lowercase. By doing this, we produce a dtm object. This has a .to_array() method which we use to represent it as a numpy array. The array is then easily converted to a dataframe; noting further that the rows will be each document (the names of which can be obtained from the original dataframe), and that the columns will be the tokens identified by the CV object. The result is displayed below.

This matrix is quite simple in that it shows us a very intutitive picture; each cell is just the number of times a word appeared in a particular document; like šusi[finger]N showed up thrice in the document labeled c.6.2.3.

In [144]:
dtm = cv.fit_transform(list(grouped.Term))
pd.DataFrame(dtm.toarray(), columns=cv.get_feature_names(), index=grouped.index)

,1-kam-ma[1st]NU,1/2[1/2]NU,1/3[1/3]NU,10-kam-ma[10th]NU,1000[1000]NU,100[100]NU,108000[108000]NU,1080[1080]NU,108[108]NU,10[10]NU,...,šusi[finger]N,šuteŋ[accept]V/t,šutubur[mixture]N,šutug[reed-hut]N,šutum[storehouse]N,šutur[garment]N,šuš[cover]V/t,šuʾi[barber]N,šuʾu[stone]N,šuʾura[goose]N
id_text,,,,,,,,,,,,,,,,,,,,,
c.0.1.1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c.0.1.2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
c.0.2.01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c.0.2.02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c.0.2.03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c.0.2.04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c.0.2.06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c.0.2.07,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c.0.2.08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


With this in hand, we then want to transform it such that we have the inverse weighting discussed above; weight the words so that words appearing more frequently aren't as big of a factor. This is accomplished by defining the transformer on the first line below, then using the transformer to transform the raw counts. 

In [16]:
tt = TfidfTransformer(use_idf=True)
dtm_tf = tt.fit_transform(dtm)
dtm_tf.shape #356 docs, 4301 unique terms, as expected

(356, 4302)

In [17]:
tfidf = pd.DataFrame(dtm_tf.toarray(), columns=cv.get_feature_names(), index=grouped.index)
tfidf

,1-kam-ma[1st]NU,1/2[1/2]NU,1/3[1/3]NU,10-kam-ma[10th]NU,1000[1000]NU,100[100]NU,108000[108000]NU,1080[1080]NU,108[108]NU,10[10]NU,...,šusi[finger]N,šuteŋ[accept]V/t,šutubur[mixture]N,šutug[reed-hut]N,šutum[storehouse]N,šutur[garment]N,šuš[cover]V/t,šuʾi[barber]N,šuʾu[stone]N,šuʾura[goose]N
id_text,,,,,,,,,,,,,,,,,,,,,
c.0.1.1,0.22871,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
c.0.1.2,0.00000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.091941,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
c.0.2.01,0.00000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
c.0.2.02,0.00000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
c.0.2.03,0.00000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
c.0.2.04,0.00000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
c.0.2.06,0.00000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
c.0.2.07,0.00000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
c.0.2.08,0.00000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


# Differences between documents

Now, with a TFIDF matrix, we can begin assessing similarity of documents by treating each row of this matrix as a respresentation of that document (specifically, as a vector) in N dimensional Euclidian space, where N is the number of features (words). Similarity of documents can be interpreted as being the distance between documents in this N space. Here, we will be interested in two measures of distance: cosine distance and Euclidian (L2) distance. 

## Cosine Distance

We start of with cosine similarity; it's a bit easier to think about. A cosine similarity (https://en.wikipedia.org/wiki/Cosine_similarity) is defined as the $cos\theta$ of two vectors, and can be calculated from the dot product of two vectors. Thus, to determine cosine similarity between two documents, we must calculate the dot products of their vector representations. 

Suppose we have a vector space with n dimensions $\mathbb{V_n}$ (think of the regular 2d Euclidian plane as  $\mathbb{V_2}$), and vectors $\boldsymbol{X},\boldsymbol{Y} \in \mathbb{V_n}$. We can represent $\boldsymbol{X}$ and $\boldsymbol{Y}$ as column vectors of size n:
$$
\boldsymbol{X} = \begin{bmatrix}
           x_{1} \\
           x_{2} \\
           \vdots \\
           x_{n}
         \end{bmatrix}, 
\boldsymbol{Y} = \begin{bmatrix}
           y_{1} \\
           y_{2} \\
           \vdots \\
           y_{n}
         \end{bmatrix}
$$

Then, the dot product of these two vectors is defined as 

$$
\boldsymbol{X}\cdot \boldsymbol{Y} = 
           x_{1}y_{1} +
           x_{2}y_{2} +
           \dots +
           x_{n}y_{n}
$$

The geometric intepretation of the dot product is defined as follows: 
$$
\boldsymbol{X}\cdot\boldsymbol{Y} = |\boldsymbol{X}||\boldsymbol{Y}|cos\theta 
$$
where $|\boldsymbol{X}|$ and $|\boldsymbol{Y}|$ are the so-called "norms" of the vectors (essentially, the lengths of the vectors). 
These norms are defined as the square root of the dot product of a vector with itself, so 
$$|\boldsymbol{X}| = \sqrt{\boldsymbol{X}\cdot \boldsymbol{X} }
           =\sqrt{x_{1}x_{1} +
           x_{2}x_{2} +
           \dots +
           x_{n}x_{n}}$$
Then, we can solve for $cos \theta$
$$
cos \theta = \frac{\boldsymbol{X}\cdot\boldsymbol{Y}}{\sqrt{|\boldsymbol{X}|}\sqrt{|\boldsymbol{Y}|}}  
$$
In other words, cosine similarity is 
$$
\frac{\boldsymbol{X}\cdot\boldsymbol{Y}}{(\boldsymbol{X}\cdot\boldsymbol{X})(\boldsymbol{Y}\cdot\boldsymbol{Y})} 
=    \frac{x_{1}y_{1} +
           x_{2}y_{2} +
           \dots +
           x_{n}y_{n}}
           {
           \sqrt{(x_{1}x_{1} +
           x_{2}x_{2} +
           \dots +
           x_{n}x_{n})}
           \sqrt{(y_{1}y_{1} +
           y_{2}y_{2} +
           \dots +
           y_{n}y_{n})}
           }
$$
Finally, **Cosine distance is simply 1 - the cosine similarity.**

To get a cosine distance, we can use scipy.spacial.disance's cosine function. It takes two arrays of the same length, and returns the cosine distance between the two:

`>> cosine(arr1, arr2)`

This returns some number between 0 and 1. Why does it return 0 and 1? Basically, you can either have two documents be completely disparate (cosine distance of 1), which comes from  or they can be completely the same (cosine distance of 0). Thinking in terms of cosine similarity, it has similarity 0 when they're disparate, and 1 when it's the same document. You can think of it mathematically in the form above; if $\boldsymbol{X}=\boldsymbol{Y}$, then the cosine is just its square divided by itself, which would yield 1. Then, it's logical that being completely dissimilar would yield 0. Of course, this also makes sense by the rule of dot products; if two vectors are orthogonal, then they have no overlap (here, overlap is highly analogous to similarity)and yield a dot product of 0. 

A quick refresher on the cosine is below.

In [18]:
#some demos
test1 = np.arange(10)
test2 = 3 * test1
test3 = -1 * test1
test4 = np.zeros(10)
test4[0] = 1
test5 = np.zeros(10)
test5[1] = 1
print(cosine(test1, test2)) #should be 1 - 1 = 0, 1 - cosine of 0
print(cosine(test1, test3)) #should be 1 - (-1) = 2, 1 - cosine of pi
print(cosine(test4, test5)) #should be 1 , 1 - cosine of pi/2

1.11022302463e-16
2.0
1.0


If the mathematical intuition doesn't make sense, here's a geometric motivation. Below is a tool to visualize cosine distance. This is for a 2d space, so basically you'd be looking at a classification between documents using just two words to differentiate them. Take a look at how the cosine similarity and cosine distance change as you change the each vector (remember, each vector represents the document). As you change the angle between them (meaning the documents are more separated, and thus more different), how does the cosine similarity and distance change? Another thing to do is to compare two situations; when a short vector and a long vector make the same angle as a long vector and a long vector. What's the relationship between the angles produced in these two cases?

In [123]:
bound = 500

In [138]:
from scipy.spatial.distance import cosine
def f(doc1word1, doc1word2, doc2word1, doc2word2):
    fig, ax = plt.subplots(figsize = (8,8))
    circle1 = plt.Circle((0, 0), bound/5, fill = False)
    arrow1 = plt.arrow(x=0, y= 0, dx = doc1word1, dy = doc1word2, width = bound/300, length_includes_head = True, color = '#003262')
    arrow2 = plt.arrow(x=0, y= 0, dx = doc2word1, dy = doc2word2, width = bound/300, length_includes_head = True, color = '#FDB515')
    ax.add_artist(circle1)
    plt.xlim(0, bound)
    plt.xlabel("tfidf of word 1 for doc")
    plt.ylabel("tfidf of word 2 for doc")
    plt.ylim(0, bound)
    plt.legend([arrow1,arrow2], ['Doc 1','Doc2'])
    plt.show()
    print("Cosine distance: ", cosine([doc1word1, doc1word2], [doc2word1, doc2word2]), "\n", 
          "Cosine similarity: ", 1 - cosine([doc1word1, doc1word2], [doc2word1, doc2word2]))

interactive_plot = interactive(f, 
                               doc1word1=widgets.FloatSlider(value=bound/2, min = 0, max = bound),
                               doc1word2=widgets.FloatSlider(value=bound/2, min = 0, max = bound),
                               doc2word1=widgets.FloatSlider(value=bound/2, min = 0, max = bound),
                               doc2word2=widgets.FloatSlider(value=bound/2, min = 0, max = bound)
                              )
# output = interactive_plot.children[-1]
# output.layout.height = '350px'
interactive_plot

You should've noticed an important feature of the cosine similarity; no matter what the lengths of the vectors are, the angle still tells us how close these two vectors look. This is really useful because it means that, to use cosine distance, we can have normalized or unnormalized data- it doesn't matter! Ie, we can directly compare short and long bodies of  text. In other cases, it might look quite different just because one has many more words than the other so will look different by some metrics (like the Euclidean distance discussed below); but cosine distance doesn't have that issue. 

As a direct demonstration on htis dataset: 
We expect the distance to be high between clusters 6 and 1, and close between documents within 6. Let's see if this reflects that.

In [140]:
print("Average cosine distance between last doc and 15 last docs:", np.mean([cosine(tfidf.iloc[-1], tfidf.iloc[-i]) for i in np.arange(1, 16)]))
print("Average cosine distance between last doc and 15 first docs: ", np.mean([cosine(tfidf.iloc[-1], tfidf.iloc[i]) for i in np.arange(15)]))

Average cosine distance between last doc and 15 last docs: 0.683795283255
Average cosine distance between last doc and 15 first docs:  0.887919647852


So looking at the average distance between the last doc (from cluster 6) and 15 other docs from cluster 6, the average distance is lower than the difference between that document and the documents from the other end, 15 docs from clusters 0 and 1. Makes sense.

## Euclidean distance

Another metric we might like to use is the Euclidian distance, which can be more natural to think about. That is, given two vectors (like the ones described above), we can compute the distance between the endpoints of those vectors as such:
$$
d = \sqrt{
(x_1 - y_1)^2 +
(x_2 - y_2)^2 + 
\dots + 
(x_n - y_n)^2 
}
$$
You might recognize this as a generalized version of the Pythagorean theorem.
Also note that this can be related to the above picture of vectors, because this exactly corresponds to the norm of the vector $\boldsymbol{X} - \boldsymbol{Y}$:
$$
|\boldsymbol{X} - \boldsymbol{Y}| = 
\Huge\lvert\small\begin{bmatrix}
x_1 - y_1 \\
x_2 - y_2 \\
\vdots\\
x_n - y_n
\end{bmatrix}\Huge\rvert
\normalsize= \sqrt{
(x_1 - y_1)^2 +
(x_2 - y_2)^2 + 
\dots + 
(x_n - y_n)^2 
}
$$
Using this definition, we can easily define a function to calculate a Euclidian distance. 

However, one caveat is that distance can be largely affected by lengths of vectors; it is apparent that vectors of distinctly different lengths will be determined to be far apart, even in the body of their text is very similar (eg, compare ['Hello World'] vs ['Hello World Hello World Hello World Hello World Hello World']). To correct for this, we can normalize the vectors (ie, divide the vector by its length, which is equivalent to dividing every single element by the vector's length/norm). All normalized vectors then have length 1. After performing this operation, the Euclidian distance can then be calculated for a more accurate idea of the distance between vectors. 

In [20]:
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import norm
print(euclidean(test1, test2))
print(euclidean(test1, test3))
print(euclidean(test4, test5))

33.7638860323
33.7638860323
1.4142135623730951


Notice that these values are too large to make sense; the furthest distance two points can be from one another on a unit sphere is 2. Normalizing will help:

In [21]:
print(euclidean(test1/norm(test1), test2/norm(test2)))
print(euclidean(test1/norm(test1), test3/norm(test3)))
print(euclidean(test4/norm(test4), test5/norm(test5)))

1.2412670766236366e-16
1.9999999999999998
1.4142135623730951


Euclidian distance can actually be safely used with tfidf matrices, as they are normalized by the idf weighting. However, it becomes more important if you are using the original term matrix with raw counts of words; the distances between documents would lbe affected by the length of documents and raw frequency of words, which is not ideal. In those cases, you would want to normalize values by dividing by length of document; ie, make it so that all documents lie on the same surface of a unit hypersphere. 

# N- grams

N grams are a way of representing groups of words as a single token. This can be useful as groups of words can sometimes be more informative than just single words themselves. For example, if you looked at the words "give" and "up" between documents, they might not tell you too much. But look at the pair of words "give up", and then you might get some greater insight. N grams are a useful tool for this reason; clusters of words can give deeper insight by considering them as a whole, rather than just by their individual parts. 

In [22]:
prefilter

,cf,extent,form,gw,id_text,lang,line_no,line_ref,pos,status,text_name,version
0,dubsaŋ,NaN,dub-saŋ-ta,first,c.0.1.1,sux,1,1,AJ,NaN,Ur III catalogue from Nibru (N1),NaN
1,Enki,NaN,{d}en-ki,1,c.0.1.1,sux,2,2,DN,NaN,Ur III catalogue from Nibru (N1),NaN
2,unu,NaN,unu₂,dwelling,c.0.1.1,sux,2,2,N,NaN,Ur III catalogue from Nibru (N1),NaN
3,gal,NaN,gal,big,c.0.1.1,sux,2,2,V/i,NaN,Ur III catalogue from Nibru (N1),NaN
4,ed,NaN,im-ed₃,ascend,c.0.1.1,sux,2,2,V/i,NaN,Ur III catalogue from Nibru (N1),NaN
5,anzag,NaN,an-zag-še₃,horizon,c.0.1.1,sux,3,3,N,NaN,Ur III catalogue from Nibru (N1),NaN
6,anŋi,NaN,an-ŋi₆,eclipse,c.0.1.1,sux,4,4,N,NaN,Ur III catalogue from Nibru (N1),NaN
7,zu,NaN,zu,know,c.0.1.1,sux,4,4,V/t,NaN,Ur III catalogue from Nibru (N1),NaN
8,ama,NaN,ama,mother,c.0.1.1,sux,4,4,N,NaN,Ur III catalogue from Nibru (N1),NaN
9,tu,NaN,tu₆,incantation,c.0.1.1,sux,4,4,N,NaN,Ur III catalogue from Nibru (N1),NaN


We group together by document, as before, this time without any filtering. 

In [146]:
grouped2 = alltex.groupby("id_text").aggregate(" ".join) #this line of code lumps together all documents' terms together, and joins them as a single space separated string
grouped2

,cf,form,gw,lang,line_no,pos,text_name,Term
id_text,,,,,,,,
c.0.1.1,dubsaŋ Enki unu ed anzag anŋi zu ama tu zu gi ...,dub-saŋ-ta {d}en-ki unu₂ im-ed₃ an-zag-še₃ an-...,first 1 dwelling ascend horizon eclipse know m...,sux sux sux sux sux sux sux sux sux sux sux su...,1 2 2 2 3 4 4 4 4 4 5 5 6 7 7 7 8 8 8 8 9 10 1...,AJ DN N V/i N N V/t N N V/t N V/t N N N V/i N ...,Ur III catalogue from Nibru (N1) Ur III catalo...,dubsaŋ[first]AJ Enki[1]DN unu[dwelling]N ed[as...
c.0.1.2,diŋir šembizida dar kur mete gud banda šerzid ...,diŋir šembi₂-zid-da dar-a nu-kur₂ me-te-bi gud...,deity kohl split different appropriate-thing o...,sux sux sux sux sux sux sux sux sux sux sux su...,1 1 1 2 3 5 5 6 7 7 7 8 9 9 10 10 10 10 11 11 ...,N N V/t V/i N N V/i N N V/i N N N V/i N N NU V...,Ur III catalogue at Yale (Y1) Ur III catalogue...,diŋir[deity]N šembizida[kohl]N dar[split]V/t k...
c.0.2.01,mi niŋdu nin me šar sud nam nun re innin me hu...,mi₂ niŋ₂-du₇-e nin me šar₂-ra su₃-ra₂-še₃ nam₂...,cvne appropriate-thing lady Being 3600 distant...,sux sux sux sux sux sux sux sux sux sux sux su...,2 3 4 4 4 5 6 6 7 8 8 8 9 9 10 11 11 12 12 13 ...,N N N N NU V/i N N DP N N V/i N V/i V/i V/i N ...,OB catalogue from Nibru (N2) OB catalogue from...,mi[cvne]N niŋdu[appropriate-thing]N nin[lady]N...
c.0.2.02,sud nam nun re innin me huš ud huš til hursaŋ ...,su₃-ra₂-še₃ nam₂ nun-e re-a in-nin me huš-a ud...,distant lord prince that lady Being reddish st...,sux sux sux sux sux sux sux sux sux sux sux su...,5 6 6 7 8 8 8 9 9 10 11 12 12 13 13 14 15 15 1...,V/i N N DP N N V/i N V/i V/i N N V/i N N DP DP...,OB catalogue in the Louvre (L) OB catalogue in...,sud[distant]V/i nam[lord]N nun[prince]N re[tha...
c.0.2.03,mu niŋul Waradsin Inab me ŋuruš šir rah enbar ...,mu-ni niŋ₂-ul-še₃ IR₃-{d}suen i₃-na-ab i-me-a ...,name everlasting 1 1 be male song beat reed fr...,sux sux sux sux sux sux sux sux sux sux sux su...,1 1 2 3 3 4 4 4 5 5 6 7 8 9 10 10 10 11 11 12 ...,N N RN SN V/i N N V/t N N N RN V/i N N N V/i Q...,OB catalogue from Urim (U1) OB catalogue from ...,mu[name]N niŋul[everlasting]N Waradsin[1]RN In...
c.0.2.04,nin mulan buršuma niŋziŋal ursaŋ mi meta di me...,nin mul-an-gin₇ bur-šum₂-ma zi-ŋal₂ ur-saŋ-me-...,lady star dowager living-creature hero cvne wh...,sux sux sux sux sux sux sux sux-x-emesal sux s...,1 1 2 3 4 5 6 6 7 7 8 8 8 9 10 10 11 11 12 13 ...,N N N N N N QP V/i QP V/i N N NU V/i J V/i V/i...,OB catalogue from Urim (U2) OB catalogue from ...,nin[lady]N mulan[star]N buršuma[dowager]N niŋz...
c.0.2.06,ursaŋ alim mah mea lu meʾam RI-RI iri re me ga...,ur-saŋ alim mah me-a lu me-a-am nu-mu-e-de₃-RI...,hero bison great where? abundant dear RI-RI ci...,sux sux sux sux sux sux sux sux sux sux sux su...,1 1 1 2 2 3 4 5 6 7 7 8 8 9 9 10 10 11 12 12 1...,N N V/i QP V/i J V N DP N V/i QP N DN DN NU V/...,OB catalogue from Nibru (N3) OB catalogue from...,ursaŋ[hero]N alim[bison]N mah[great]V/i mea[wh...
c.0.2.07,uru2 er balaŋ Nin.X illalum aya illalum balaŋ ...,uru₂ am₃-ma-er₂-ra-bi balaŋ {d}nin-X il-la-lum...,town tears instrument 1 expression cry express...,sux-x-emesal sux sux sux sux sux sux sux sux s...,A1 A1 A2 A2 A4 A4 A4 A5 A5 A6 A6 A6 A7 A7 A7 A...,N N N DN J N J N DN N V/i N NU N DN NU N NU N ...,OB catalogue possibly from Zimbir (B1) OB cata...,uru2[town]N er[tears]N balaŋ[instrument]N Nin....
c.0.2.08,a-ce-er il nir atuku gel-le-ej3 mah uma gig um...,a-še-er il₂ nir a₂-tuku gel-le-eŋ₃-ŋa₂-ŋu₁₀ ma...,lament raise trust powerful to-be-lost great t...,sux-x-emesal sux sux sux sux-x-emesal sux sux ...,A4 A6 B3 B3 B4 B6 B7 B7 B9 B9 B10 B10 B11 B11 ...,N V/t N N V V/i N V/i N V/t N V/i DN N V/i DN ...,OB catalogue from Nibru (N4) OB catalogue from...,a-ce-er[lament]N il[raise]V/t nir[trust]N atuk...


We define a count vectorizer as before, but this time, we define an ngram range to produce ngrams of n = 1 and 2.

In [149]:
cv2 = CountVectorizer(token_pattern='[^ ]+', lowercase = False, ngram_range=(1, 2)) #initialize a count vectorizer object with correct tokenization for this term format

In [185]:
dtm2 = cv2.fit_transform(list(grouped2.Term))
dtm2df = pd.DataFrame(dtm2.toarray(), columns=cv2.get_feature_names(), index=grouped2.index)
dtm2df

,1-kam-ma[1st]NU,1-kam-ma[1st]NU 1[1]NU,1-kam-ma[1st]NU Agade[1]SN,1-kam-ma[1st]NU Awan[1]SN,1-kam-ma[1st]NU Hamazi[1]SN,1-kam-ma[1st]NU Mašgula[1]PN,1-kam-ma[1st]NU Ninŋišzidak[1]DN,1-kam-ma[1st]NU a[arm]N,1-kam-ma[1st]NU amaš[sheepfold]N,1-kam-ma[1st]NU aya[cry]N,...,šuʾu[stone]N,šuʾu[stone]N bar[outside]V/t,šuʾu[stone]N dugud[heavy]V/i,šuʾu[stone]N gasura[stone]N,šuʾu[stone]N murgu[shoulder]N,šuʾu[stone]N saŋkal[stone]N,šuʾu[stone]N tukul[weapon]N,šuʾura[goose]N,šuʾura[goose]N a[arm]N,šuʾura[goose]N bazbaz[bird]N
id_text,,,,,,,,,,,,,,,,,,,,,
c.0.1.1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c.0.1.2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c.0.2.01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c.0.2.02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c.0.2.03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c.0.2.04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c.0.2.06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c.0.2.07,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c.0.2.08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


This output might seem a little confusing, so take a look at the feature names used by the count vectorizer:

In [166]:
cv2.get_feature_names()[44000:45000]

['namtar[fate]N zid[right]V/i',
 'namtar[fate]N zu[know]V/t',
 'namtar[fate]N ŋeš[tree]N',
 'namte[fear]N',
 'namte[fear]N isiš[tears]N',
 'namte[fear]N nam[fate]N',
 'namtibira[sculpture]N',
 'namtibira[sculpture]N de[bring]V/t',
 'namtibira[sculpture]N namdubsar[the-scribal-arts]N',
 'namtibira[sculpture]N šum[give]V/t',
 'namtil[life]N',
 'namtil[life]N 1[1]NU',
 'namtil[life]N Eunir[1]TN',
 'namtil[life]N KI.LUGAL.DU[locus]N',
 'namtil[life]N Marduk[1]DN',
 'namtil[life]N Ninurtak[1]DN',
 'namtil[life]N Ninŋišzidak[1]DN',
 'namtil[life]N Siniddinam[1]RN',
 'namtil[life]N Ziudsura[1]RN',
 'namtil[life]N a[arm]N',
 'namtil[life]N a[water]N',
 'namtil[life]N abba[father]N',
 'namtil[life]N abula[gate]N',
 'namtil[life]N aya[father]N',
 'namtil[life]N aŋ[measure]V/t',
 'namtil[life]N ba[garment]N',
 'namtil[life]N bar[outside]V/t',
 'namtil[life]N dari[eternal]V/i',
 'namtil[life]N de[bring]V/t',
 'namtil[life]N dirig[float]V/i',
 'namtil[life]N du[push]V/t',
 'namtil[life]N du[spread]

It's now easier to see that we indeed have generated 1 and 2 grams! The column headings have just put the second term on a separate line from the first term in 2 grams. We can now potentially do some nice things with this, but first we have to check for the stuff that we filtered out in the first half; bigrams with those won't be terribly useful, as they aren't real information (things like NA, (X)).

So we generate a list of columns where we have these nonsense words, and drop them.

In [186]:
nonsense = ["[nan]nan" in i or "[X](X)" in i for i in cv2.get_feature_names()]
dtm2df = dtm2df.drop(np.arange(len(nonsense))[nonsense])
dtm2df

,1-kam-ma[1st]NU,1-kam-ma[1st]NU 1[1]NU,1-kam-ma[1st]NU Agade[1]SN,1-kam-ma[1st]NU Awan[1]SN,1-kam-ma[1st]NU Hamazi[1]SN,1-kam-ma[1st]NU Mašgula[1]PN,1-kam-ma[1st]NU Ninŋišzidak[1]DN,1-kam-ma[1st]NU a[arm]N,1-kam-ma[1st]NU amaš[sheepfold]N,1-kam-ma[1st]NU aya[cry]N,...,šuʾu[stone]N,šuʾu[stone]N bar[outside]V/t,šuʾu[stone]N dugud[heavy]V/i,šuʾu[stone]N gasura[stone]N,šuʾu[stone]N murgu[shoulder]N,šuʾu[stone]N saŋkal[stone]N,šuʾu[stone]N tukul[weapon]N,šuʾura[goose]N,šuʾura[goose]N a[arm]N,šuʾura[goose]N bazbaz[bird]N
id_text,,,,,,,,,,,,,,,,,,,,,
c.0.1.1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c.0.1.2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c.0.2.01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c.0.2.02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c.0.2.03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c.0.2.04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c.0.2.06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c.0.2.07,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c.0.2.08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
